In [2]:
import numpy as np
from scipy.linalg import expm,logm

from scipy.integrate import solve_ivp
from scipy.integrate import quad

# Define matrices A and B
A = np.array([[2, 0], [0, 3]])
B = np.array([[5, 6], [7, 8]])

# Calculate commutator of A and B
commutator = np.dot(A, B) - np.dot(B, A)

# Print commutator
print("The commutator of A and B is:")
print(commutator)


The commutator of A and B is:
[[ 0 -6]
 [ 7  0]]


In [3]:
x=np.array([[5],[-1]])
x2=np.array([[2],[3]])

In [4]:
y=np.dot(A,x)
y

array([[10],
       [-3]])

## Fonctions utiles

In [5]:
import numpy as np

def mult(*matrices):
    if len(matrices) == 0:
        raise ValueError("At least one matrix is required.")
        
    # Check that all matrices have compatible dimensions
    shape = matrices[0].shape
    for i in range(1, len(matrices)):
        if shape[1] != matrices[i].shape[0]:
            raise ValueError("Matrices are not compatible for multiplication.")

    product = matrices[0]
    for i in range(1, len(matrices)):
        product = np.dot(product, matrices[i])

    return product

def elem(n,i,j):
    E = np.zeros((n, n))
    E[i, j] = 1
    return E

def comm(A,B):
    return np.dot(A, B) - np.dot(B, A)

def exp(A):
    exp_A = np.eye(A.shape[0])
    A_k=A
    fact=1
    num_terms = 20
    for i in range(1, num_terms+1):
        term = A_k/fact 
        exp_A += term
        A_k=mult(A_k,A)
        fact*=(i+1)
    return exp_A


def log(A):
    log_A=np.eye(A.shape[0])
    A_k=A
    fact=-1

In [6]:
elem(3,0,1)

array([[0., 1., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [7]:
mult(A,B,x)

array([[38],
       [81]])

In [8]:
def grad_exp(A,i,j):
    n=len(A)
    term1=expm(A)
    term2=elem(n,i,j)
    term3=0.5*comm(A,term2)
    term4=comm(A,term3)/3
    matrix_sum=term4+term2+term2
    return mult(term1,matrix_sum)

In [9]:
def grad_norm_i_j(A,x,y,i,j):
    exp_A=expm(A)
    exp_A_t=expm(A.T)
    grad_exp_A=grad_exp(A,i,j)
    grad_exp_A_t=grad_exp(A.T,i,j)
    term_1=mult(x.T,grad_exp_A_t,exp_A,x)
    term_2=mult(x.T,exp_A_t,grad_exp_A,x)
    term_3=-mult(y.T,grad_exp_A,x)
    term_4=-mult(x.T,grad_exp_A_t,y)
    return float(term_1+term_2+term_3+term_4)

In [10]:
grad_norm_i_j(A,x,y,1,0)

-4890.321236443212

In [11]:
def grad_norm_ij(A,x_list,y_list,i,j):
    N=len(x_list)
    return 1/N*sum([grad_norm_i_j(A,x,y,i,j) for x, y in zip(x_list, y_list)])

In [12]:
def grad_matrix(A,x_list,y_list):
    dA_dt = np.zeros_like(A.copy())
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            dA_dt[i][j] = -grad_norm_ij(A, x_list,y_list, i, j)   
    return dA_dt

In [13]:
np.array([[0,1],[1,2]])

array([[0, 1],
       [1, 2]])

## Calculating A(1)

In [14]:
def grad_norm_total(t, A_flat, x_list,y_list):
    n=2
    A_flat=np.array(A_flat.copy())
    A = A_flat.reshape((n, n))
    
    gradient = grad_matrix(A,x_list,y_list)
    return gradient.flatten()

def evolve_matrix(A_0, x_list,y_list, t_span, t_target):
    n = 2
    A_0_flat = A_0.flatten()
    
    t_eval = np.array([t_target])
    
    # Solve the differential equation using solve_ivp
    solution = solve_ivp(grad_norm_total, t_span, A_0_flat, t_eval=t_eval, args=(x_list,y_list))
    
    A_t = solution.y.reshape((n, n))
    
    return A_t

In [15]:
np.dot(A,[x,x2])

array([[[10],
        [ 4]],

       [[-3],
        [ 9]]])

In [16]:
# Define initial matrix and list of vectors
A = np.array([[2, 0], [0, 3]])
A_0=np.ones((2,2))
x_list = [x,x2]
y_list=np.dot(A,x_list)

# Evolve the matrix to time t=0.5
t_span = (0, 1)
t_target = 1
A_t = evolve_matrix(A_0, x_list,y_list, t_span, t_target)

# Print the resulting matrix
print("Matrix at time t="+str(t_target))
print(A_t)

Matrix at time t=1
[[0.09984018 0.650636  ]
 [0.3238767  0.5113399 ]]


In [17]:
expm(A_t)

array([[1.24161311, 0.92091404],
       [0.458417  , 1.82405225]])

In [18]:
print("Matrix at time t=0.5:")
print(A_t)

Matrix at time t=0.5:
[[0.09984018 0.650636  ]
 [0.3238767  0.5113399 ]]


In [19]:
A_t.shape[0]

2

## Nouvelle approche

In [20]:
def grad_W_kp(A,k,p):
    n = A.shape[0]
    result = np.zeros((n, n))
    At=A.T
    V=elem(n,k,p)
    for i in range(n):
        for j in range(n):
            def integrand(t, i, j, A, V):
                expAt = expm(A*t)
                expA1t = expm(A*(1-t))
                return expA1t[i,:] @ V @ expAt[:,j]
            
            result[i,j], _ = quad(integrand, 0, 1, args=(i, j, At, V))
    return result

In [21]:
grad_W_kp(A,0,1)

array([[ 0.        , 12.69648082],
       [ 0.        ,  0.        ]])

In [22]:
def grad_norm(A,x,y):
    n=A.shape[0]
    W=expm(A)
    grad=0
    for k in range(n):
        for p in range(n):
            grad_w=grad_W_kp(A,k,p)
            grad+=2*x[p]*(W[k][p]*x[p]-y[k])*grad_w
    return grad

In [23]:
expm(A)[0][1]

0.0

In [24]:
grad_norm(A,x,y)

array([[1991.00189176,  253.92961649],
       [ 380.89442473,  686.34436545]])

In [25]:
np.eye(2)

array([[1., 0.],
       [0., 1.]])

In [26]:
A0 = np.eye(2)  # initial condition
t0 = 0    # initial time
dt = 0.01  # time step
t_max = 0.5  # maximum time to integrate to

A = A0.copy()  # make a copy of A0 to store the result

while t0 < t_max:
    grad = grad_norm(A, x, y)
    A -= dt*grad
    t0 += dt
print(A)

[[ 0.27829202  1.24926679]
 [ 0.84233785 -3.26550334]]


In [27]:
expm(A)

array([[1.62475819, 0.52196457],
       [0.35194285, 0.14410118]])

In [35]:
print(y_list[0])
print(A@x_list[0])

[[1.91335005]
 [0.7951758 ]
 [1.37596613]
 [1.46677149]
 [0.87349634]]
[[1.91335005]
 [0.7951758 ]
 [1.37596613]
 [1.46677149]
 [0.87349634]]


In [37]:
np.linalg.norm(A-A_est)

1.1470728710031e-13

In [151]:
import numpy as np

# Generate data
n = 1000  # number of data points
d = 5  # dimension of x and y
A0 = np.random.rand(d, d)
A=expm(A0.copy())
x = np.random.rand(n, d)
noise = 0.05 * np.random.randn(n, 1)
y = np.dot(x, A.T)+noise

# Estimate A
X = np.hstack((x, np.ones((n, 1))))  # augment X with a column of ones for the intercept
A_est, _, _, _ = np.linalg.lstsq(X, y, rcond=None)
A_est = A_est[:-1, :]  # remove the last row (corresponding to the intercept)

# Print the original and estimated matrices
print("Original matrix A:")
print(A)
print("Estimated matrix A_est:")
print(A_est.T)

Original matrix A:
[[4.06970107 1.49055831 3.11045163 2.04775071 3.23793727]
 [4.42126777 4.04782603 4.31152807 2.84825267 3.96878304]
 [3.03461035 2.14951158 4.65381005 2.83340567 3.41429412]
 [2.10776208 1.74288257 2.58698756 2.88527886 2.19732441]
 [2.63765156 1.48725303 2.64720175 1.98644125 3.12278354]]
Estimated matrix A_est:
[[4.06880539 1.49270251 3.11485499 2.05598342 3.23474099]
 [4.42037208 4.04997023 4.31593142 2.85648538 3.96558677]
 [3.03371466 2.15165578 4.6582134  2.84163838 3.41109785]
 [2.10686639 1.74502677 2.59139092 2.89351158 2.19412813]
 [2.63675587 1.48939723 2.65160511 1.99467397 3.11958727]]


In [129]:
x

array([[4.24443728e-01, 1.61846624e-01, 4.67447828e-01, 7.69945237e-01,
        6.54357604e-01],
       [5.93954023e-03, 6.27217055e-01, 8.52590675e-01, 8.32678069e-01,
        7.68166573e-01],
       [3.53616336e-01, 4.86436613e-01, 3.23135959e-01, 4.65533074e-01,
        3.29859231e-01],
       [5.67145686e-01, 4.76929478e-03, 8.66873460e-02, 4.18150087e-01,
        5.71239342e-01],
       [1.39241928e-01, 7.41146613e-01, 1.59645896e-01, 2.11196097e-01,
        5.87955635e-02],
       [4.67280859e-01, 7.88304906e-01, 3.76390074e-01, 5.84398785e-01,
        5.16842829e-02],
       [7.78617823e-01, 3.37323638e-01, 3.99888614e-01, 8.23492995e-01,
        7.46736603e-01],
       [8.54632771e-01, 8.63429227e-01, 3.30439532e-01, 7.29247584e-02,
        7.67194231e-01],
       [9.15372026e-01, 3.95837655e-01, 7.84514279e-01, 1.38643036e-04,
        4.77799655e-01],
       [5.94704724e-01, 8.09756305e-01, 3.86480663e-01, 5.27488791e-01,
        3.46531749e-01]])

In [137]:
np.vstack([x,x[0]])

array([[4.24443728e-01, 1.61846624e-01, 4.67447828e-01, 7.69945237e-01,
        6.54357604e-01],
       [5.93954023e-03, 6.27217055e-01, 8.52590675e-01, 8.32678069e-01,
        7.68166573e-01],
       [3.53616336e-01, 4.86436613e-01, 3.23135959e-01, 4.65533074e-01,
        3.29859231e-01],
       [5.67145686e-01, 4.76929478e-03, 8.66873460e-02, 4.18150087e-01,
        5.71239342e-01],
       [1.39241928e-01, 7.41146613e-01, 1.59645896e-01, 2.11196097e-01,
        5.87955635e-02],
       [4.67280859e-01, 7.88304906e-01, 3.76390074e-01, 5.84398785e-01,
        5.16842829e-02],
       [7.78617823e-01, 3.37323638e-01, 3.99888614e-01, 8.23492995e-01,
        7.46736603e-01],
       [8.54632771e-01, 8.63429227e-01, 3.30439532e-01, 7.29247584e-02,
        7.67194231e-01],
       [9.15372026e-01, 3.95837655e-01, 7.84514279e-01, 1.38643036e-04,
        4.77799655e-01],
       [5.94704724e-01, 8.09756305e-01, 3.86480663e-01, 5.27488791e-01,
        3.46531749e-01],
       [4.24443728e-01, 1.6184

In [106]:
print("Original matrix A:")
print(A0)
print("Estimated matrix A_est:")
print(logm(A_est.T))

Original matrix A:
[[0.12401276 0.2666253  0.66931183 0.07988674 0.61478305]
 [0.07672272 0.67273915 0.33377838 0.14814857 0.70640233]
 [0.02771108 0.26825821 0.35309527 0.65225814 0.39419931]
 [0.812989   0.72111458 0.45265439 0.97037084 0.57763803]
 [0.1224104  0.73489164 0.18646502 0.82825933 0.35366912]]
Estimated matrix A_est:
[[0.12401276 0.2666253  0.66931183 0.07988674 0.61478305]
 [0.07672272 0.67273915 0.33377838 0.14814857 0.70640233]
 [0.02771108 0.26825821 0.35309527 0.65225814 0.39419931]
 [0.812989   0.72111458 0.45265439 0.97037084 0.57763803]
 [0.1224104  0.73489164 0.18646502 0.82825933 0.35366912]]


In [74]:
y_list=np.dot(A,x_list)
y_list

array([[[0.75375832],
        [0.26810454],
        [0.42952112],
        [1.31901245],
        [0.58996115]],

       [[0.92029545],
        [0.38019473],
        [0.56636916],
        [1.62905632],
        [1.13545806]],

       [[1.6848453 ],
        [1.01240581],
        [0.97998871],
        [1.45282654],
        [1.28122186]],

       [[1.39025112],
        [0.52091128],
        [0.71732578],
        [1.97732859],
        [1.15563354]],

       [[1.71512799],
        [0.803281  ],
        [0.85958091],
        [1.59022358],
        [1.10402646]]])

In [78]:
np.linalg.inv(sum([np.dot(x,x.T) for x in x_list]))

array([[ 51.94157412, -40.25110447, -63.64526369,  34.67397926,
         36.26133545],
       [-40.25110447,  36.86638902,  52.75444309, -32.54413305,
        -30.73405007],
       [-63.64526369,  52.75444309,  83.57420034, -48.63536997,
        -47.64903189],
       [ 34.67397926, -32.54413305, -48.63536997,  31.80442534,
         27.66502724],
       [ 36.26133545, -30.73405007, -47.64903189,  27.66502724,
         28.00874501]])

In [68]:
y_noisy

array([[[0.27750522],
        [0.28563166],
        [0.30099303],
        [0.48742153],
        [0.41423892]],

       [[0.79574625],
        [0.97930198],
        [1.07431012],
        [1.38061654],
        [1.24105594]],

       [[1.04207073],
        [1.06276523],
        [1.67728438],
        [1.60488074],
        [1.67605013]],

       [[0.34364134],
        [0.54138324],
        [0.73841344],
        [0.61460121],
        [0.62998633]],

       [[0.69486995],
        [0.6836659 ],
        [1.52798652],
        [1.07978987],
        [1.39276923]],

       [[0.41299999],
        [0.69414139],
        [0.90674194],
        [0.55744358],
        [0.55948162]],

       [[0.94079216],
        [1.01113912],
        [1.21774696],
        [1.5612865 ],
        [1.47641556]],

       [[0.71853466],
        [0.6307787 ],
        [1.34888327],
        [1.22442592],
        [1.45264368]],

       [[1.098966  ],
        [1.16221403],
        [1.72522229],
        [1.70446433],
        [1.75376

In [65]:
[np.random.rand(5, 1) for i in range(100)]

array([[0.26182721, 0.85875066, 0.80009154, 0.34084328, 0.10985421],
       [0.87728085, 0.79888206, 0.053852  , 0.25724297, 0.58172919],
       [0.58153434, 0.84917968, 0.56715161, 0.66552427, 0.66247143],
       [0.74222989, 0.0449112 , 0.55826061, 0.74814736, 0.70214628],
       [0.61818615, 0.90286317, 0.99914865, 0.18621455, 0.39668641]])

## Dynamique de convergence avec le gradient flow

### Vérification numérique de $A=S X^{-1}$

In [53]:
def compute_S(x_list,y_list):
    return sum([np.dot(y,x.T) for (x,y) in zip(x_list,y_list)])

def compute_X(x_list):
    return sum([np.dot(x,x.T) for x in x_list])

In [54]:
import numpy as np

# Generate some noisy data
A = np.random.rand(5, 5)
print(A)
x_list = [np.random.rand(5, 1) for i in range(10000)]
y_list = [np.dot(A,x) for x in x_list] 

S=compute_S(x_list,y_list)
X_inv=np.linalg.inv(compute_X(x_list))

A_est=np.dot(S,X_inv)
print(A_est)
print("norme de frobenius : ",np.linalg.norm(A-A_est))

[[0.96062218 0.56127466 0.78004136 0.53619539 0.72720491]
 [0.1793138  0.26522296 0.04220734 0.00111306 0.25418089]
 [0.1086676  0.97294059 0.53597269 0.94944892 0.50788828]
 [0.12501343 0.52745795 0.14274215 0.85167235 0.99950626]
 [0.26227071 0.86716405 0.36075835 0.44481751 0.16195542]]
[[0.96062218 0.56127466 0.78004136 0.53619539 0.72720491]
 [0.1793138  0.26522296 0.04220734 0.00111306 0.25418089]
 [0.1086676  0.97294059 0.53597269 0.94944892 0.50788828]
 [0.12501343 0.52745795 0.14274215 0.85167235 0.99950626]
 [0.26227071 0.86716405 0.36075835 0.44481751 0.16195542]]
norme de frobenius :  1.0866619140072263e-13


In [76]:
from scipy.integrate import solve_ivp

n=5
# Définir les matrices X et S
X = compute_X(x_list)
S = compute_S(x_list,y_list)

dt = 0.01

# Définir le nombre d'itérations
num_iterations = 10

# Initialiser W
W = np.eye(n)


# Itérer pour résoudre l'équation différentielle
for i in range(num_iterations):  
    print(i)
    W += (-np.dot(W,X) + S) * dt

0
1
2
3
4
5
6
7
8
9
